# 3D Stellar wind

---
In this example, we consider an smoothed-particle hydrodynamics (SPH) model of a companion-perturbed stellar wind.

In [15]:
import numpy             as np
import matplotlib.pyplot as plt
import os
from astropy import units, constants


---

## Model setup
First we download a snapshot of this SPH model.

In [13]:
setup_file = '3D_stellar_wind_data/wind.setup'
input_file = '3D_stellar_wind_data/wind.in'
dump_file  = '3D_stellar_wind_data/wind.dump'

In [14]:
!wget 'https://raw.githubusercontent.com/Ensor-code/phantom-models/main/Malfait%2B2024a/v10e00/wind.setup'  --output-document $setup_file
!wget 'https://raw.githubusercontent.com/Ensor-code/phantom-models/main/Malfait%2B2024a/v10e00/wind.in'     --output-document $input_file
!wget 'https://raw.githubusercontent.com/Ensor-code/phantom-models/main/Malfait%2B2024a/v10e00/wind_v10e00' --output-document $dump_file

--2024-08-08 16:03:19--  https://raw.githubusercontent.com/Ensor-code/phantom-models/main/Malfait%2B2024a/v10e00/wind.setup
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1209 (1.2K) [text/plain]
Saving to: ‘3D_stellar_wind_data/wind.setup’

3D_stellar_wind_dat 100%[===================>]   1.18K  --.-KB/s    in 0s      

2024-08-08 16:03:19 (30.7 MB/s) - ‘3D_stellar_wind_data/wind.setup’ saved [1209/1209]

--2024-08-08 16:03:20--  https://raw.githubusercontent.com/Ensor-code/phantom-models/main/Malfait%2B2024a/v10e00/wind.in
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP reques

We use [plons](https://github.com/Ensor-code/plons) to open the data.

In [10]:
import plons

setupData = plons.LoadSetup(f'{os.getcwd()}/3D_stellar_wind_data', 'wind')
dumpData  = plons.LoadFullDump(f'{os.getcwd()}/{dump_file}', setupData)

position = dumpData["position"]*1e-2   # position vectors [cm   -> m]
velocity = dumpData["velocity"]*1e3    # velocity vectors [km/s -> m/s]
rho      = dumpData["rho"]             # density          [g/cm^3]
tmp      = dumpData["Tgas"]            # temperature      [K]
tmp[tmp<2.7] = 2.7                     # Cut-off temperatures below 2.7 K

# Unpack velocities
v_x, v_y, v_z = velocity.T

# Convert rho (total density) to H2 abundance
nH2 = rho * 1.0e+6 * constants.N_A.si.value / 2.02

# Define turbulence at 150 m/s
trb = 150.0

Next, we map the particle data to a regular Cartesian mesh.

In [11]:
from pomme.haar import Haar

# Map point data to a regular grid
haar = Haar(position, q=8)
# Zoom in on the centre region to avoid edge effects
imin = 2**(haar.q-3)
imax = 3*imin
# Map data to a regular grid
nH2_dat = haar.map_data(nH2, interpolate=True)[-1][imin:imax,imin:imax,imin:imax]
tmp_dat = haar.map_data(tmp, interpolate=True)[-1][imin:imax,imin:imax,imin:imax]
v_x_dat = haar.map_data(v_x, interpolate=True)[-1][imin:imax,imin:imax,imin:imax]
v_y_dat = haar.map_data(v_y, interpolate=True)[-1][imin:imax,imin:imax,imin:imax]
v_z_dat = haar.map_data(v_z, interpolate=True)[-1][imin:imax,imin:imax,imin:imax]

---

### TensorModel
With all the data in place, we can start building a pomme model.
First, we store all model parameters as a TensorModel object and store this in an HDF5 file.
We will use this later as the ground truth to verify our reconstructions against.

In [ ]:
from pomme.model import TensorModel

model = TensorModel(shape=nH2_dat.shape, sizes=haar.xyz_L)
model['log_H2'          ] = np.log(nH2_dat).astype(np.float64)
model['log_temperature' ] = np.log(tmp_dat).astype(np.float64)
model['velocity_x'      ] =        v_x_dat .astype(np.float64)
model['velocity_y'      ] =        v_y_dat .astype(np.float64)
model['velocity_z'      ] =        v_z_dat .astype(np.float64)
model['log_v_turbulence'] = np.log(trb)*np.ones(model.shape, dtype=np.float64)
model.save('3D_stellar_wind_truth.h5')

---

### GeneralModel
First, we define the functions that can generate the model distributions from the model parameters.

In [ ]:
import torch
from pomme.utils import planck, T_CMB

def get_velocity(model):
    """
    Get the velocity from the TensorModel.
    """
    return model['velocity_z']

def get_temperature(model):
    """
    Get the temperature from the TensorModel.
    """
    return torch.exp(model['log_temperature'])

def get_abundance(model, l):
    """
    Get the abundance from the TensorModel.
    """
    # Define the assumed molecular fractions w.r.t. H2
    X_mol = [3.0e-4, 5.0e-6]
    # Return the abundance
    return torch.exp(model['log_H2']) * X_mol[l]

def get_turbulence(model):
    """
    Get the turbulence from the TensorModel.
    """
    return torch.exp(model['log_v_turbulence'])

def get_boundary_condition(model, frequency):
    """
    Get the boundary condition from the TensorModel.
    model: TensorModel
        The TensorModel object containing the model.
    frequency: float
        Frequency at which to evaluate the boundary condition.
    """
    # Compute the incoming boundary intensity
    Ibdy  = torch.ones((model.shape[0], model.shape[1], len(frequency)), dtype=torch.float64)
    Ibdy *= planck(temperature=T_CMB, frequency=frequency)
    # Return the result
    return Ibdy

Using these functions, we can build a GeneralModel object that can be used to generate synthetic observations or reconstruct the required parameters. (Cfr. the SphericalModel class for spherically symmetric models.)

In [ ]:
from pomme.model import GeneralModel

gmodel = GeneralModel(model=model)
gmodel.get_velocity           = get_velocity
gmodel.get_abundance          = get_abundance
gmodel.get_turbulence         = get_turbulence
gmodel.get_temperature        = get_temperature
gmodel.get_boundary_condition = get_boundary_condition

---

### Spectral lines
We base our reconstructions on synthetic observations of two commonly observed rotational lines CO $J=4-3$ and SiO $J=3-2$. We explicitly provide the molar mass for SiO, since this is not extracted correctly from the line data file.

In [16]:
from pomme.lines import Line
from pomme.utils import get_molar_mass

lines = [
    Line(species_name='CO',     transition=3),
    Line(species_name='sio-h2', transition=2, molar_mass=get_molar_mass('SiO'))
]

You have selected line:
    CO(J=4-3)
Please check the properties that were inferred:
    Frequency         4.610407682e+11  Hz
    Einstein A coeff  6.126000000e-06  1/s
    Molar mass        28.0101          g/mol
You have selected line:
    sio-h2(J=03-02)
Please check the properties that were inferred:
    Frequency         1.302686830e+11  Hz
    Einstein A coeff  1.058000000e-04  1/s
    Molar mass        44.0849          g/mol


/home/frederikd/.local/lib/python3.9/site-packages/astroquery/lamda/core.py:145: UserWarning: The first time a LAMDA function is called, it must assemble a list of valid molecules and URLs.  This list will be cached so future operations will be faster.
  warnings.warn("The first time a LAMDA function is called, it must "


---

### Frequencies
Next, we define the velocity/frequency range.
We observe the lines in 100 frequency bins, centred around the lines, with a spacing of 120 m/s.

In [ ]:
vdiff = 120   # velocity increment size [m/s]
nfreq = 100   # number of frequencies

velocities  = nfreq * vdiff * torch.linspace(-1, +1, nfreq, dtype=torch.float64)
frequencies = [(1.0 + velocities / constants.c.si.value) * line.frequency for line in lines]

---

## Synthetic observations
We can now generate synthetic observations, directly from the Model object.
We will use these later to derive our reconstructions.